# Preparing the enviroment and mounting data

In [1]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
#from google.colab import drive
#drive.mount("/content/drive/")
#cd drive/My Drive/SFU/Project4-Spring2021
!ls

data_preprossing_functions.py  known_snps.txt  x_clinical_names.npy  x_snps.npy
eval.py			       sample_data     x_clinical.npy	     y.npy
known_snps_fullname.txt        train.py        x_colnames.npy


In [2]:
!git clone https://github.com/JakeColtman/bartpy.git
#can i just clone parkca? check the differences between my train and the one on the package

Cloning into 'bartpy'...
remote: Enumerating objects: 1971, done.
remote: Total 1971 (delta 0), reused 0 (delta 0), pack-reused 1971
Receiving objects: 100% (1971/1971), 11.97 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (1329/1329), done.


In [3]:
# reading snps file
#libraries
import pandas as pd
import numpy as np 
from scipy import sparse
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import sys
sys.path.insert(0,'bartpy/')
from bartpy.sklearnmodel import SklearnModel

preprocessing = False
run_learners = False

# BCCH Data Analysis 

## Data 
There are two options: 
1. Pre-processing: require original files on the server and perform the pre-processing / filtering 
2. Load files saved on local machine 


In [15]:
path = '/content/'
if preprocessing:  
  print('Option 1: Starting pre-processing:')
  import data_preprossing_functions as dpf
  y, x_clinical, x_snps, x_colnames, x_clinical_names = dpf.run_preprocessing(path, True)
  x_clinical = x_clinical.astype('float64')
  print('Done!')
  print('Shapes:',len(y), sum(y), x_clinical.shape, x_snps.shape, len(x_colnames), len(x_clinical_names))
  #318 (318, 42) (318, 16641) 16641
else: 
  print('Option 2: Reading files')
  y =  np.load('y.npy')
  x_clinical = np.load('x_clinical.npy',allow_pickle =True)
  x_clinical = x_clinical.astype('float64')
  x_clinical_names = np.load('x_clinical_names.npy',allow_pickle =True)
  x_snps = np.load('x_snps.npy',allow_pickle =True)
  x_colnames  = np.load('x_colnames.npy',allow_pickle =True)
  print('Done!')
  print('Shapes:',len(y), sum(y), x_clinical.shape, x_snps.shape, len(x_colnames), len(x_clinical_names))
 

Option 2: Reading files
Done!
Shapes: 302 187 (302, 41) (302, 16195) 16195 41


## ParKCa

1. Learners: Deconfounder, BART, CEVAE

BART OR CEVAE CODE? 



In [23]:
if run_learners:
  import train as parkca
  level1data = parkca.learners_bcch(path_output = path, DA = True, BART = True, X = x_snps, y = y, 
                                    colnamesX = x_colnames,causes = 'snps', Z = x_clinical, colnamesZ = x_clinical_names)

X = np.concatenate([x_clinical,x_snps], axis = 1)
print(X.shape)

X = preprocessing.StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


(302, 16236)


In [ ]:
print('...Learner: BART')
model = SklearnModel(n_trees = 50,n_burn = 50, n_chains=1, n_jobs=1 ) # Use default parameters
model.fit(X_train, y_train) # Fit the model
print('......making predictions')
y_train_pred = model.predict(X_train) # [:,0:1000] Make predictions on the train set
y_test_pred = model.predict(X_test) # [:,0:1000] Make predictions on the train set

def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
    https://stackoverflow.com/questions/28719067/roc-curve-and-cut-off-point-python
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

...Learner: BART


  0%|          | 0/50 [00:00<?, ?it/s]

Starting burn


  2%|▏         | 1/50 [00:07<05:48,  7.10s/it]

In [ ]:
thhold = Find_Optimal_Cutoff(y_train,y_train_pred)
y_train_pred01 = [0 if item < thhold else 1 for item in y_train_pred]
y_test_pred01 = [0 if item < thhold else 1 for item in y_test_pred]
print('\n...Training set: F1 - ',f1_score(y_train,y_train_pred01))
print('...... confusion matrix: ',confusion_matrix(y_train,y_train_pred01).ravel())

print('\n...Testing set: F1 - ',f1_score(y_test,y_test_pred01))
print('...... confusion matrix: ',confusion_matrix(y_test,y_test_pred01).ravel())


In [ ]:
#very bad on testing 